In [ ]:
import numpy as np
import scipy as sp
import matplotlib.pyplot as plt

In [ ]:
e = 1.602176634e-19
hbar = 1.0545718e-34

In [ ]:
nu = 15E9
n = 20
gamma = 0.0146

file = "../data/iv_0.05.dat"
V0,I0 = np.loadtxt(file, unpack=True)

In [ ]:
def findVal(x: np.ndarray, y:np.ndarray, val):
    for i,valx in enumerate(x):
        if valx == val:
            return y[i]
        elif valx < val:
            continue
        elif valx > val:
            x_c, y_c = x[i], y[i]
            if i <= 0:
                return y[0]
            x_p, y_p = x[i-1], y[i-1]
            x_d = x_c - x_p
            y_d = y_c - y_p

            return y_p + y_d/x_d * (val - x_p)
    return y[-1]

def IV(v0,vw):
    w = 2*np.pi*nu
    vw = vw * gamma
    b_arg = (e*vw)/(hbar*w)
    out = 0.0
    for ni in range(-n,n+1):
        bessel = (sp.special.jv(ni,b_arg))**2
        v_shift = v0 - (ni*hbar*w)/(e)
        v_shift = np.abs(v_shift)
        I = findVal(V0,I0,v_shift)
        out += I * bessel
    return out

In [ ]:
x = V0
y = np.arange(0.0, 4.0, 0.1)
y *= 1e-3

z = np.zeros((len(x), len(y)), dtype=np.float64)

for yi, yval in enumerate(y):
    print("\r"+" "* 80, end="")
    print("\rCalculating IV curve: %d/%d" % (yi+1, len(y)), end="")
    for xi, xval in enumerate(x):
        z[xi][yi] = IV(xval,yval)
print()

dz = np.zeros((len(x)-1, len(y)))
for yi in range(len(y)):
    dz[:,yi] = np.diff(z[:,yi])/np.diff(x)

In [ ]:
# show z[1]
plt.plot(x, z[:,5], label="new")
plt.plot(V0, I0, label="old")
plt.legend()
plt.show()

# show dz[1]
plt.plot(x[:-1], dz[:,5], label="new")
plt.plot(V0[:-1], np.diff(I0)/np.diff(V0), label="old")
plt.legend()
plt.show()

In [ ]:
dz_t = np.transpose(dz)
dz_t = np.flipud(dz_t)
plt.imshow(
    dz_t, 
    aspect="auto",
    extent=(x[0], x[-1], y[0], y[-1]),
    )
plt.colorbar()
plt.show()